#Conectamos a google drive e importamos librerias

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd
!pip install pandas_summary
from pandas_summary import DataFrameSummary
import numpy as np
import datetime
!pip install isoweek
from isoweek import Week
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
!pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from random import sample

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

# Para importar el Helper
%cd "/gdrive/Shareddrives/Redes neuronales/TP2"
!pip install import-ipynb
import import_ipynb
from Helper import *

/gdrive/Shareddrives/Redes neuronales/TP2
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=0868bb5c6de1917f9663ca9b98dc43faae0033de0f7eda2617d8f468b5e0623b
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from Helper.ipynb


# Se cargan los archivos preprocesados

In [3]:
df = pd.read_feather('data preprecesada/train_normalized_data_!sales0.fth')
df_test = pd.read_feather('data preprecesada/test_normalized_data.fth')

In [4]:
len(df.columns)

89

In [5]:
df.columns

Index(['index', 'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week',
       'Day', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitatio

# Listamos las variables

##Variables categóricas

In [6]:
 cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day',
             'StateHoliday', 'SchoolHoliday', 'Promo2SinceWeek',
             'CompetitionMonthsOpen', 'Promo2Weeks', 'Promo2',
             'StoreType', 'Assortment', 'PromoInterval', 
             'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 'Week',
              'Promo', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw',
             'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw',
             'Close_bool_bw','Close_bool_fw']

In [7]:
np.unique(df['Promo'])

array([0, 1])

## Variables continuas

In [8]:
contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 
               'Min_TemperatureC', 'Precipitationmm',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool',
    'AfterPromo','BeforePromo',
   'StateHoliday_bool','AfterSchoolHoliday','BeforeSchoolHoliday',
   'AfterClose_bool','BeforeClose_bool']

In [9]:
len(contin_vars)+len(cat_vars)

50

In [10]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]
uniques

,Store,DayOfWeek,Year,Month,Day,StateHoliday,SchoolHoliday,Promo2SinceWeek,CompetitionMonthsOpen,Promo2Weeks,Promo2,StoreType,Assortment,PromoInterval,CompetitionOpenSinceYear,Promo2SinceYear,State,Week,Promo,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw,Close_bool_bw,Close_bool_fw
uniques,1115,7,3,12,31,4,2,24,25,26,2,4,3,4,21,8,12,52,2,22,6,6,3,3,8,8,7,7


# Corremos los modelos

##Asignación de embeddings

In [11]:
np.unique(df['Promo'])

array([0, 1])

In [12]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 3,'Promo':2, 'Year': 3, 'Month': 4,
'Day': 10, 'StateHoliday': 3,'SchoolHoliday':2, 'Promo2SinceWeek': 3,
'CompetitionMonthsOpen': 4, 'Promo2Weeks': 3,  'Promo2':2, 'StoreType': 2,
 'Assortment': 3, 'PromoInterval': 3,'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 
 'State': 4, 'Events': 7, 'Promo_fw': 4,'Promo_bw': 4, 'Week':10,
 'StateHoliday_bool_fw': 2, 'StateHoliday_bool_bw': 2, 'SchoolHoliday_fw': 2,
'SchoolHoliday_bw': 2,'Close_bool_bw':3,'Close_bool_fw':3}

In [13]:
chosen =['Store','Day']
values_chosen = [cat_var_dict[i] for i in chosen]
dictionary = dict(zip(chosen,values_chosen))
dictionary

{'Day': 10, 'Store': 50}

In [14]:
len(np.unique(df['Store']))

1115

In [15]:
np.random.randint(1,2)

1

In [16]:
np.random.randint(1,len(np.unique(df['Store'])))

865

##Modelo

In [17]:
# Parametros de entrada para creación de las capas ocultas:
cant_capas_ocultas = 2
cantidad_neuronas = [1000, 500]
l2_lambda = 1e-3
activation_function = ["relu","relu"]
alpha = [0, 0] # Se puede cambiar el valor de alpha en caso de usar función LeakyRelu.
# Recordar que el alpha tiene que ser un vector
kernel_initializer = "uniform"

# Parametros de entrada para creación modelo entero:
add_customers = True
output_activation = 'linear'

all_vars = cat_vars + contin_vars #variables elegidas

log_output = False

lr=0.001

metrics=['mse', rmspe]
loss='mse'
epochs = 10
batch_size = 256
verbose=2

df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

Hacemos que le entren 10 características categóricas y 10 características continuas por lo menos. 

In [18]:
cat_vars_random = [sample(cat_vars,np.random.randint(10,len(cat_vars))) for i in range(3)]

In [19]:
contin_vars_random = [sample(contin_vars,np.random.randint(10,len(contin_vars))) for i in range(3)]

In [20]:
res_metrics = pd.DataFrame(columns=['Cantidad de características','Caracteristicas modelo','Puntaje (Sales_rmspe)'])
for i in cat_vars_random:

  cat_vars_chosen = i

  print(f'Cat vars chosen: {cat_vars_chosen}')
  cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model2(cat_vars_chosen, uniques, cat_var_dict)

  for j in contin_vars_random:

    contin_vars_chosen = j 

    print(f'Cont vars chosen: {contin_vars_chosen}')
    cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars_chosen)

    print("Creamos los embeddings y layers")
    concatenate_layer = MLP_layers(cat_vars_embed_outs, cont_vars_inputs, l2_lambda, kernel_initializer, cant_capas_ocultas, cant_neuronas= cantidad_neuronas, activation_func= activation_function)
    print(concatenate_layer)

    print(f'Creamos el modelo {len(res_metrics)}')
    print(cat_var_inputs)
    model = crear_MLP(add_customers, cat_var_inputs, cont_vars_inputs, concatenate_layer, output_activation)

    all_vars = cat_vars_chosen + contin_vars_chosen

    print("Normalizamos el dataset")
    X_train,X_val,X_test,y_train,y_val,y_norm = obtener_x_y(df_train,df_val,df_test,all_vars,add_customers=add_customers,log_output=log_output)  

    model_chosen=f'modelos generados/eleccion caracteristicas/modelos entrenados/bestmodel7-{len(res_metrics)}.hdf5'

    print(f'Entrenamos el modelo {len(res_metrics)}')
    history,model = entrenar_MLP(X_train,X_val,y_train,y_val,all_vars,model=model,add_customers=add_customers,log_output=log_output,lr=lr,metrics=metrics,loss=loss,model_chosen=model_chosen,epochs = epochs,batch_size = batch_size,verbose=verbose)  

    print("Obtenemos las métricas")
    metric_cross = metricas_MLP(X_val,y_val,model,model_chosen=model_chosen)    
    puntaje = metric_cross['Sales_rmspe']

    data_columns_df = dict(zip(res_metrics.columns,[len(all_vars),all_vars,puntaje]))
    res_metrics = res_metrics.append(data_columns_df,ignore_index=True)

res_metrics.to_csv("/gdrive/Shareddrives/Redes neuronales/TP2/eleccion caracteristicas/resultados/results_eleccion7.csv")

Cat vars chosen: ['Promo2SinceYear', 'CompetitionOpenSinceYear', 'StateHoliday_bool_fw', 'StateHoliday', 'StoreType', 'SchoolHoliday_fw', 'Year', 'Promo', 'DayOfWeek', 'CompetitionMonthsOpen', 'Promo2SinceWeek']
Cont vars chosen: ['StateHoliday_bool', 'BeforeStateHoliday_bool', 'BeforeSchoolHoliday', 'AfterPromo', 'AfterSchoolHoliday', 'trend_DE', 'Max_Humidity', 'Min_Humidity', 'Max_TemperatureC', 'Mean_Wind_SpeedKm_h', 'AfterStateHoliday_bool', 'CloudCover', 'Precipitationmm', 'Min_TemperatureC', 'Mean_TemperatureC', 'Mean_Humidity', 'AfterClose_bool', 'CompetitionDistance']
Creamos los embeddings y layers
KerasTensor(type_spec=TensorSpec(shape=(None, 500), dtype=tf.float32, name=None), name='activation_1/Relu:0', description="created by layer 'activation_1'")
Creamos el modelo 0
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'Promo2SinceYear_input')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'CompetitionOpenSinceYear_input')>, <KerasTensor: shape